# A general plan
* Explore metrics used to evaluate conversational flow
* Choose an open-source model that 
    1. fits my hardware capacity
    2. has nice metrics
* Choose conversational datasets to finetune
* Prepare said datasets
* Add LoRA code - thanks to yandexdataschool for additional guidance!
* Finetune the model (using huggingface trainer)

...then wrap the model into a container

## Explore metrics
Naive, regarding interaction statistics
* Length of conversation: how long (in user lines) lasts a chat on average.
* Time between replies to the chatbot, which could indicate the level of a user's engagement. Requires analytics to establish thresholds.
* Human feedback, as in
    - "Were you satisfied by the conversation? Y/N" at the end of the conversation - does not give fine-grained information about what happened during the conversation that led to a particular assessment
    - grade 1-10 - just as above. 
    - This is overall messy, but it makes sense for such feedback to be collected, then post-analyzed by assessors.
* How often do users come back to chat again? Need users!
* How often is a chat with a model abandoned at the start? Need users!


Naive, based on text
* repetition / fluency, as in N of distinct n-grams - the only quantifiable metric. However, it really is just a proxy metric for generation quality; does not measure the adequacy of the conversation...
* perplexity on the user questions? - this is not about answers' quality at all; besides, I doubt that huge models have difficulties in this domain
* overlap between a user's question and a model's answer - I think this is a bit outdated for LLM evaluation, since it used to be a problem around 2017-2020 maybe. Could make sense if we have enough time to check the LM's answer before outputting it to the user. I incorporated it, anyway. 
* F-measure/BLEU/etc. on questions with pre-defined answers? - might be good for tracking factual information, but the good chit-chat has nothing pre-defined a priori.
* Embedding distance between the user's question and the model's answer - seems pretty much the same as the previous idea
* % of negative user responses ("no, that's not what I was talking about"; "that's a dumb response"; "bad bot"); % of positive user responses ("You're funny!", "Great, thanks", "you're right")
    - very hard to define 'negative', but theoretically it could be (embedding) cosine similarity to sets of responses similar to above; results in a problem of constructing such sets, extracting user responses from paragraphs, etc.
    - metrics such as these are really very much user-dependent (what if a user is often ironic? what if they're displaying opinions far from a bot's training data distribution? what if a user is in a bad mood...)
    - but __could be captured__ if reframed as an entailment task (LM-user answer pairs, entailment/contradiction/neutral; we're looking for contradictions; something like https://arxiv.org/abs/1904.03371 ?) or as a sentiment analysis task, but requires additional models

I actually expect that interaction-based metrics per model are much more telling than the textual ones.

I looked at several dialogue evaluation datasets, and found them either too specialized ([MultiWOZ](https://github.com/budzianowski/multiwoz/tree/master/data/MultiWOZ_2.2)) or too vague for usage in automatic evaluation (([OpenDialKG](https://github.com/facebookresearch/opendialkg/tree/main)) - this one looks good for fine-tuning, though!)

There are attempts at building specialized metrics involving various aspects of quality, e.g. empathy: https://github.com/Sea94/ieval

And we could always try and evaluate on some task that involves generating capabilities under a restriction, e.g. a narrowly defined task like summarization in terms of sentence compression. Unfortunately, in causal LMs, that involves heavy amount of prompting, and if a metric depends on a prompt engineer skill (while prompts are uncomparable across different models), it does not look pretty.

I checked https://github.com/violet-zct/fairseq-detect-hallucination/ in order to see for myself how this 'hallucination' metric worked - this framework does not use metrics like ROUGE or BLEU. It is not defined for LMs, however.

### To sum up
This is a real headscratcher. Human evaluation, with an addition of user interaction-based numbers, should probably do the work. 

## Choose a model

A recent causal LM that fits my memory is all I ask, so under 13B
* OPT? Comes in flavours like < 1B, 1.3B, 2.7B, 6.7B
* GPT-J? 6B
* LLaMA? 7B

I looked at https://weightwatcher.ai/leaderboard.html and https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard and compared the top models regarding their Alpha and Average, respectively. Frankly, these two metrics seem to be contradicting each other. 

Surprisingly, an OPT-1.3B model was pretty small, high in Alpha and decent in terms of Truthfulness, although other metrics were not as impressive. Alas, my RAM is limited, and so I could not build an image using anything really large.

Since I haven't had any opinion based on experience, I decided to test the pipeline on a (really) small OPT. 

If I had enough time/memory, I'd switch to the fresh LLaMA-2 (7B) -- it demonstrates a good average on the leaderboard tasks.



In [3]:
# %pip install transformers datasets peft accelerate bitsandbytes sacremoses pandas 

In [5]:
from transformers import AutoTokenizer, AutoConfig

# model_id = "facebook/opt-1.3b"
model_id = 'facebook/opt-350m'

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
# max_length is not set properly
max_len = AutoConfig.from_pretrained(model_id).max_position_embeddings
tokenizer.model_max_length = max_len

## Choose datasets & prepare them

Personally, I think that all dialogue corpora ought to be curated. As a source of data, Reddit probably contains most lively responses, but it is potentially unpredictable and should be tested first. I considered the following datasets:

* ConvAI2/3 dataset - on the closer look, it has a deeper structure that is useful in RLHF or evaluation contexts
* The NPS Chat Corpus - not in Huggingface Datasets, but potentially useful
* Cornell Movie-Dialogs Corpus
* HC3 (which also contains ChatGPT answers, but we won't need them)
* and DailyDialogue corpus


And then the problems started.

- I haven't opened a PR yet, but the builder script for 'cornell_movie_dialog' is broken
    
    `y = load_dataset('cornell_movie_dialog')`
- 'Hello-SimpleAI/HC3' is broken too
    
    `z = load_dataset('Hello-SimpleAI/HC3')`
- The NPS Chat Corpus is a part of NLTK, and it's completely unusable because it is a loose collection of posts and not (somewhat coherent) dialogs. 


I was left with DailyDialogue and manually-processed HC3.

By the way, these should've worked better if the data also contained prompts (like 'USER1 says '), in a manner in which the chatbot is prompted (see src/chatbot.py)

In [6]:
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
# by the way, daily_dialogs is suddenly pre-tokenized in some places
from sacremoses import MosesDetokenizer

dd = load_dataset('daily_dialog') # ['dialog']
detok = MosesDetokenizer('en')

# I really admire this line in tokenization utils:
#     if max_length is None and len(ids) > self.model_max_length and verbose: <some warning about how it's not going to use max_length>
# why declaring it then?

def process_daily_dialogs(examples):
    temp = [[detok.detokenize(x.split()) for x in y] for y in examples['dialog']]
    temp = [' | '.join(x) for x in temp]
    examples = tokenizer(temp, max_length=tokenizer.model_max_length)
    return examples

dd['train'] = dd['train'].map(process_daily_dialogs, batched=True, remove_columns=["dialog", 'act', 'emotion'])
dd['test'] = dd['test'].map(process_daily_dialogs, batched=True, remove_columns=["dialog", 'act', 'emotion'])

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
# !wget https://huggingface.co/datasets/Hello-SimpleAI/HC3/resolve/main/all.jsonl
import json
import pandas as pd
from datasets import Dataset


def load_hc3():
    with open("all.jsonl") as inp:
        data = pd.DataFrame.from_records([json.loads(line) for line in inp])
    return data

hc3 = Dataset.from_pandas(load_hc3())
hc3 = hc3.train_test_split(test_size=0.1)

def process_hc3(examples):
    # This could have been done in pandas, I think...
    pairs = []
    for x, y in zip(examples['question'], examples['human_answers']):
        x = [detok.detokenize(x.split())] * len(y)
        y = [detok.detokenize(t.split()) for t in y]
        pairs.extend([' | '.join(d) for d in zip(x, y)])
    examples = tokenizer(pairs, max_length=tokenizer.model_max_length)
    return examples
    
hc3['train'] = hc3['train'].map(process_hc3, batched=True, remove_columns=["question", 'human_answers', 'chatgpt_answers', 'index', 'source'])
hc3['test'] = hc3['test'].map(process_hc3, batched=True, remove_columns=["question", 'human_answers', 'chatgpt_answers', 'index', 'source'])

Map:   0%|          | 0/21889 [00:00<?, ? examples/s]

Map:   0%|          | 0/2433 [00:00<?, ? examples/s]

In [8]:
from datasets import concatenate_datasets

data_train = concatenate_datasets([dd['train'], hc3['train']]) 
data_test = concatenate_datasets([dd['test'], hc3['test']]) 

## LoRA

I'll use one PEFT-LoRa example, because I don't really have a lot of time to look deeper than that. I suppose that Prefix Tuning could work better, though.

In [ ]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# Can't load back quantization without GPU 
# from peft import prepare_model_for_int8_training
# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map={"":0})
# model = prepare_model_for_int8_training(model)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map='cpu')  #load_in_8bit=True
model.gradient_checkpointing_enable()

config = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj", "k_proj", "v_proj"], 
                    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

model = get_peft_model(model, config)
model = model.float()
# model = model.to('cpu')

Some weights of OPTForCausalLM were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
print(model.state_dict()['base_model.model.lm_head.weight'].sum(axis=0))

tensor([ 3.4723e+02,  1.5523e+03, -3.5968e+03,  8.8323e+02, -1.1636e+02,
         5.8384e+02, -2.2802e+02,  2.1153e+02,  4.7491e+02,  1.7522e+03,
         4.3498e+02,  6.0625e+02,  1.0186e+02,  1.8586e+02, -3.1153e+02,
         2.0905e+02, -3.6086e-01, -1.0476e+03, -3.3628e+03, -7.7822e+01,
        -5.2680e+03, -3.1091e+02,  1.5983e+02, -1.7921e+02, -5.0815e+02,
         3.9251e+02, -1.9007e+03,  9.6294e+02, -2.5768e+02, -1.2473e+02,
        -5.9086e+01, -1.4384e+02, -2.0451e+03, -2.6408e+03,  3.1167e+03,
         9.2373e+01, -2.7629e+02,  2.1598e+03,  2.5048e+03,  1.6986e+00,
        -5.5232e+02,  1.3826e+02, -2.6508e+02, -4.9032e+03,  3.4328e+02,
         1.4574e+03, -2.2531e+02, -1.8587e+03, -2.6004e+02,  9.2003e+01,
         1.6195e+02, -5.1665e+03, -3.5247e+03, -2.2519e+03, -9.1076e+01,
        -7.3918e+02,  2.3497e+02, -2.7405e+02, -2.8799e+01,  4.6253e+02,
        -1.8389e+02,  5.5446e+02, -1.2216e+02,  6.4612e+02,  1.0152e+03,
        -4.5776e+02, -8.4575e+01,  3.5000e+03, -2.6

## Combine everything

In [21]:
from transformers import Trainer, TrainingArguments

loader = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [22]:
args = TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        auto_find_batch_size=False,
        # warmup_steps=250,
        # max_steps=5000,
        warmup_steps=1,
        max_steps=2,
        learning_rate=2e-4,  # ?
        no_cuda=True, 
        logging_steps=250,
        output_dir="outputs",
        # optim="paged_adamw_8bit"
        optim='adamw_torch'
    )

trainer = Trainer(
    model=model,
    train_dataset=data_train,
    eval_dataset=data_test,
    args=args,
    data_collator=loader
)

trainer.train()
trainer.save_model('weights')

  0%|          | 0/2 [00:00<?, ?it/s]

{'train_runtime': 12.7164, 'train_samples_per_second': 0.157, 'train_steps_per_second': 0.157, 'train_loss': 3.553908348083496, 'epoch': 0.0}


In [ ]:
from peft import PeftConfig, PeftModel

peft_config = PeftConfig.from_pretrained('weights')
model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path,
                                             # load_in_8bit=True, 
                                             device_map='cpu')
model = PeftModel.from_pretrained(model, 'weights')

Some weights of OPTForCausalLM were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
print(model.state_dict()['base_model.model.lm_head.weight'].sum(axis=0))  # hmmm... something fishy, OPT-1.3B didn't have this warning

tensor([ 3.4723e+02,  1.5523e+03, -3.5968e+03,  8.8323e+02, -1.1636e+02,
         5.8384e+02, -2.2802e+02,  2.1153e+02,  4.7491e+02,  1.7522e+03,
         4.3498e+02,  6.0625e+02,  1.0186e+02,  1.8586e+02, -3.1153e+02,
         2.0905e+02, -3.6086e-01, -1.0476e+03, -3.3628e+03, -7.7822e+01,
        -5.2680e+03, -3.1091e+02,  1.5983e+02, -1.7921e+02, -5.0815e+02,
         3.9251e+02, -1.9007e+03,  9.6294e+02, -2.5768e+02, -1.2473e+02,
        -5.9086e+01, -1.4384e+02, -2.0451e+03, -2.6408e+03,  3.1167e+03,
         9.2373e+01, -2.7629e+02,  2.1598e+03,  2.5048e+03,  1.6986e+00,
        -5.5232e+02,  1.3826e+02, -2.6508e+02, -4.9032e+03,  3.4328e+02,
         1.4574e+03, -2.2531e+02, -1.8587e+03, -2.6004e+02,  9.2003e+01,
         1.6195e+02, -5.1665e+03, -3.5247e+03, -2.2519e+03, -9.1076e+01,
        -7.3918e+02,  2.3497e+02, -2.7405e+02, -2.8799e+01,  4.6253e+02,
        -1.8389e+02,  5.5446e+02, -1.2216e+02,  6.4612e+02,  1.0152e+03,
        -4.5776e+02, -8.4575e+01,  3.5000e+03, -2.6

# Evaluation